In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np

import importlib
import work
import work.models as m
import work.data as d
import utils
import utils.image_manipulator as im
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.optimizers import RMSprop, Adam
import yaml    
import logging
import keras
from matplotlib import pyplot as plt
import random
import wandb
import copy
from wandb.keras import WandbMetricsLogger

from keras.initializers import Constant
from keras.layers import Input, Conv2D, Flatten, Activation, MaxPool2D, Dropout
from keras.models import Model

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

import tensorflow as tf
from keras import backend as K
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from tensorflow.keras.optimizers import SGD

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import work.models as m
import work.data as d


with open(r'config.yaml') as file:
    cfg = yaml.load(file, Loader=yaml.FullLoader)
    BASE_PATH = cfg['BASE_PATH']
    IMG_WIDTH = cfg['IMG_WIDTH']
    IMG_HEIGHT = cfg['IMG_HEIGHT']


logger = logging.getLogger('main')
logger.setLevel(logging.INFO)
ch = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s(%(name)s) %(levelname)s: %(message)s')
ch.setFormatter(formatter)
ch.setLevel(logging.DEBUG)
if (logger.hasHandlers()):
    logger.handlers.clear()
  
logger.addHandler(ch)

 
BASE_FILE_PATH = BASE_PATH + 'baza5.csv'
TRAIN_PATH = BASE_PATH + 'modeling/all_images/train/'
VAL_PATH = BASE_PATH + 'modeling/all_images/validation/'
TEST_PATH = BASE_PATH + 'modeling/all_images/test/'
INPUT_PATH = BASE_PATH + 'modeling/all_images/'
OUTPUT_TEST_PATH = TEST_PATH
OUTPUT_TRAIN_PATH = TRAIN_PATH
OUTPUT_VAL_PATH = VAL_PATH

def model_cnn1(_img_width, _img_height):
    model = Sequential()
    model.add(Input(shape=(_img_height, _img_width, 1)))
    model.add(Conv2D(8, (3, 3), padding="same", activation="relu"))
    model.add(MaxPool2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(12, (3, 3), padding="same", activation="relu"))
    model.add(MaxPool2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(16, (3, 3), padding="same", activation="relu"))
    model.add(MaxPool2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(24, (3, 3), padding="same", activation="relu"))
    model.add(MaxPool2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
    model.add(MaxPool2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(48, (3, 3), padding="same", activation="relu"))
    model.add(Dropout(0.5))
    model.add(Conv2D(filters=1,kernel_size=(5, 5),kernel_initializer="glorot_normal",bias_initializer=Constant(value=-0.9)))
    model.add(Flatten())
    model.add(Dense(2, activation='sigmoid'))
    
    return model

# def data():
#     (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
#     X_train, X_val, y_train, y_val = train_test_split(X_train,    y_train, test_size=0.2, random_state=12345)
#     X_train = X_train.reshape(48000, 784)
#     X_val = X_val.reshape(12000, 784)
#     X_train = X_train.astype('float32')
#     X_val = X_val.astype('float32')
#     X_train /= 255
#     X_val /= 255
#     nb_classes = 10
#     Y_train = np_utils.to_categorical(y_train, nb_classes)
#     Y_val = np_utils.to_categorical(y_val, nb_classes)
#     return X_train, Y_train, X_val, Y_val

def model(X_train, Y_train, X_val, Y_val):
    
    model = Sequential()
    model.add(Input(shape=(180, 180, 1)))
    model.add(Dense({{choice([128, 256, 512, 1024])}}, input_shape=(784,)))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([128, 256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(10))
    model.add(Activation('softmax'))
    adam = keras.optimizers.Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    rmsprop = keras.optimizers.RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    sgd = keras.optimizers.SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
   
    choiceval = {{choice(['adam', 'sgd', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = adam
    elif choiceval == 'rmsprop':
        optim = rmsprop
    else:
        optim = sgd
        
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=optim)
    model.fit(X_train, Y_train,
              batch_size={{choice([128,256,512])}},
              nb_epoch=20,
              verbose=2,
              validation_data=(X_val, Y_val))
    score, acc = model.evaluate(X_val, Y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


def data():
    with open(r'config.yaml') as file:
        cfg = yaml.load(file, Loader=yaml.FullLoader)
        BASE_PATH = cfg['BASE_PATH']
        IMG_WIDTH = cfg['IMG_WIDTH']
        IMG_HEIGHT = cfg['IMG_HEIGHT']
    
    BASE_FILE_PATH = BASE_PATH + 'baza5.csv'
    TRAIN_PATH = BASE_PATH + 'modeling/all_images/train/'
    VAL_PATH = BASE_PATH + 'modeling/all_images/validation/'
    TEST_PATH = BASE_PATH + 'modeling/all_images/test/'
    INPUT_PATH = BASE_PATH + 'modeling/all_images/'
    OUTPUT_TEST_PATH = TEST_PATH
    OUTPUT_TRAIN_PATH = TRAIN_PATH
    OUTPUT_VAL_PATH = VAL_PATH

    d.split_files(INPUT_PATH, OUTPUT_TRAIN_PATH, OUTPUT_VAL_PATH, OUTPUT_TEST_PATH, 0.15, 0.15)

    X_train, y_train, X_val, y_val, X_test, y_test = d.split_data(BASE_FILE_PATH, OUTPUT_TRAIN_PATH, OUTPUT_VAL_PATH, OUTPUT_TEST_PATH, 0, 'none')
    return X_train, y_train, X_val, y_val


In [2]:

best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      trials=Trials(),
                                     notebook_name='sample')

>>> Imports:
#coding=utf-8

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import cv2
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import importlib
except:
    pass

try:
    import work
except:
    pass

try:
    import work.models as m
except:
    pass

try:
    import work.data as d
except:
    pass

try:
    import utils
except:
    pass

try:
    import utils.image_manipulator as im
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from tensorflow.keras.optimizers import RMSprop, Adam
except:
    pass

try:
    import yaml
except:
    pass

try:
    import logging
except:
    pass

try:
    import keras
except:
    pass

try:
    from matplotlib import pyplot as plt
except:
    pass

try:
    import random
except:
    pass

try:
    import wandb
except:
    pass

try:
    import copy
except:
    pass



AttributeError: 'numpy.random.mtrand.RandomState' object has no attribute 'integers'